### Initialization

Let's first set up the python environment and define the configuration file

In [10]:
# Import required standard modules
import shutil
import sys
from pathlib import Path

import numpy as np

# Import required icepy4d4D modules
from icepy4d import classes as icepy4d_classes
from icepy4d import matching
from icepy4d import sfm
from icepy4d import io
from icepy4d import utils as icepy4d_utils
from icepy4d.classes.epoch import Epoch, Epoches
from icepy4d.metashape import metashape as MS
from icepy4d.utils import initialization as inizialization

In [11]:
# Define the path to the configuration file
CFG_FILE = "config/config_2022.yaml"

Inizialize all the required variables

In [12]:
# Parse the configuration file
cfg_file = Path(CFG_FILE)
cfg = inizialization.parse_cfg(cfg_file)

# Initialize the timer and logger
timer_global = icepy4d_utils.AverageTimer()
logger = icepy4d_utils.get_logger()

# Get the list of cameras from the configuration file
cams = cfg.cams

# Get the list of images from the configuration file
images, epoch_dict = inizialization.initialize_image_ds(cfg)

# Initialize an empty Epoches object to store the results of each epoch
epoches = Epoches(starting_epoch=cfg.proc.epoch_to_process[0])


ICEpy4D
Image-based Continuos monitoring of glaciers' Evolution with low-cost stereo-cameras and Deep Learning photogrammetry
2023 - Francesco Ioli - francesco.ioli@polimi.it

2023-08-29 17:07:45 | [INFO    ] Configuration file: config_2022
2023-08-29 17:07:45 | [INFO    ] Configuration file: config_2022
2023-08-29 17:07:45 | [INFO    ] Epoch_to_process set to a pair of values. Expanding it for a range of epoches from epoch 0 to 158.
2023-08-29 17:07:45 | [INFO    ] Epoch_to_process set to a pair of values. Expanding it for a range of epoches from epoch 0 to 158.
2023-08-29 17:07:45 | [INFO    ] Image datastores created successfully.
2023-08-29 17:07:45 | [INFO    ] Image datastores created successfully.


### Stereo Processing

The stereo processing is carried out for each epoch in order to find matched features, estimating camera pose, and triangulating the 3D points. 
The output of this step is a set of 3D points and their corresponding descriptors.

The processing for all the epoches is then iterated in a big loop.

#### Load or create a new Epoch object

In [14]:
# Initialize a timer to measure the processing time
timer = icepy4d_utils.AverageTimer()

# Get epoch id to process
ep = cfg.proc.epoch_to_process[0]

# Define paths to the epoch directory and the matching directory
epochdir = cfg.paths.results_dir / epoch_dict[ep]
match_dir = epochdir / "matching"

# Load an existing epoch or create a new one
if cfg.proc.load_existing_results:
    try:
        # Load existing epcoh from pickle file
        epoch = Epoch.read_pickle(epochdir / f"{epoch_dict[ep]}.pickle")

    except:
        logger.error(
            f"Unable to load epoch {epoch_dict[ep]} from pickle file. Creating new epoch..."
        )
        epoch = inizialization.initialize_epoch(
            cfg=cfg, images=images, epoch_id=ep, epoch_dir=epochdir
        )

else:
    # Create new epoch object
    epoch = inizialization.initialize_epoch(
        cfg=cfg, images=images, epoch_id=ep, epoch_dir=epochdir
    )

#### Feature matching with SuperGlue

In [18]:
# Define matching parameters
matching_quality = matching.Quality.HIGH
tile_selection = matching.TileSelection.PRESELECTION
tiling_grid = [4, 3]
tiling_overlap = 200
geometric_verification = matching.GeometricVerification.PYDEGENSAC
geometric_verification_threshold = 1
geometric_verification_confidence = 0.9999

# Create a new matcher object
matcher = matching.SuperGlueMatcher(cfg.matching)
matcher.match(
    epoch.images[cams[0]].value,
    epoch.images[cams[1]].value,
    quality=matching_quality,
    tile_selection=tile_selection,
    grid=tiling_grid,
    overlap=tiling_overlap,
    do_viz_matches=True,
    do_viz_tiles=False,
    save_dir=match_dir,
    geometric_verification=geometric_verification,
    threshold=geometric_verification_threshold,
    confidence=geometric_verification_confidence,
)
timer.update("matching")

2023-08-29 17:08:26 | [INFO    ] Running inference on device cuda
2023-08-29 17:08:26 | [INFO    ] Running inference on device cuda
Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)
2023-08-29 17:08:28 | [INFO    ] Matching by tiles...
2023-08-29 17:08:28 | [INFO    ] Matching by tiles...
2023-08-29 17:08:28 | [INFO    ] Matching tiles by preselection tile selection
2023-08-29 17:08:28 | [INFO    ] Matching tiles by preselection tile selection
2023-08-29 17:08:30 | [INFO    ] Matching completed.
2023-08-29 17:08:30 | [INFO    ] Matching completed.
2023-08-29 17:08:30 | [INFO    ]  - Matching tile pair (3, 2)
2023-08-29 17:08:30 | [INFO    ]  - Matching tile pair (3, 2)
2023-08-29 17:08:33 | [INFO    ]  - Matching tile pair (4, 7)
2023-08-29 17:08:33 | [INFO    ]  - Matching tile pair (4, 7)
2023-08-29 17:08:35 | [INFO    ]  - Matching tile pair (5, 7)
2023-08-29 17:08:35 | [INFO    ]  - Matching tile pair (5, 7)
2023-08-29 17:08:37 | [INFO    ]  - Matching tile pair (5

True

Extract the matched features from the Matcher object and save them in the current Epoch object

In [21]:
# Define a dictionary with empty Features objects for each camera
f = {cam: icepy4d_classes.Features() for cam in cams}

# Stack matched keypoints, descriptors and scores into Features objects
f[cams[0]].append_features_from_numpy(
    x=matcher.mkpts0[:, 0],
    y=matcher.mkpts0[:, 1],
    descr=matcher.descriptors0,
    scores=matcher.scores0,
)
f[cams[1]].append_features_from_numpy(
    x=matcher.mkpts1[:, 0],
    y=matcher.mkpts1[:, 1],
    descr=matcher.descriptors1,
    scores=matcher.scores1,
)

# Store the dictionary with the features in the Epoch object
epoch.features = f

#### Scene reconstruction

First, perform Relative orientation of the two cameras by using the matched features and the a-priori camera interior orientation.

In [22]:
# Initialize RelativeOrientation class with a list containing the two
# cameras and a list contaning the matched features location on each camera.
relative_ori = sfm.RelativeOrientation(
    [epoch.cameras[cams[0]], epoch.cameras[cams[1]]],
    [
        epoch.features[cams[0]].kpts_to_numpy(),
        epoch.features[cams[1]].kpts_to_numpy(),
    ],
)
relative_ori.estimate_pose(
    threshold=cfg.matching.pydegensac_threshold,
    confidence=0.999999,
    scale_factor=np.linalg.norm(
        cfg.georef.camera_centers_world[0] - cfg.georef.camera_centers_world[1]
    ),
)
# Store result in camera 1 object
epoch.cameras[cams[1]] = relative_ori.cameras[1]

2023-08-29 17:22:05 | [INFO    ] Relative Orientation - valid points: 1840/2090
2023-08-29 17:22:05 | [INFO    ] Relative Orientation - valid points: 1840/2090
2023-08-29 17:22:05 | [INFO    ] Relative orientation Succeded.
2023-08-29 17:22:05 | [INFO    ] Relative orientation Succeded.


Triangulate points into the object space

In [23]:
triang = sfm.Triangulate(
    [epoch.cameras[cams[0]], epoch.cameras[cams[1]]],
    [
        epoch.features[cams[0]].kpts_to_numpy(),
        epoch.features[cams[1]].kpts_to_numpy(),
    ],
)
points3d = triang.triangulate_two_views(
    compute_colors=True, image=images[cams[1]].read_image(ep).value, cam_id=1
)

2023-08-29 17:23:01 | [INFO    ] Point triangulation succeded: 1.0.
2023-08-29 17:23:01 | [INFO    ] Point triangulation succeded: 1.0.
2023-08-29 17:23:01 | [INFO    ] Point colors interpolated
2023-08-29 17:23:01 | [INFO    ] Point colors interpolated


#### Big loop over the epoches

Stack all the processing of a single epoch into a function and iterate over all the epoches


In [ ]:
# Define processing for single epoch
def process_epoch():
    pass

In [ ]:
# Add epoch to epoches object
epoches.add_epoch(epoch)

In [ ]:
logger.info("------------------------------------------------------")
logger.info("Processing started:")
timer = icepy4d_utils.AverageTimer()
iter = 0  # necessary only for printing the number of processed iteration
